# Gradient Boosting

In [1825]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn import preprocessing


In [1826]:
loan_df=pd.read_csv('loan_train.csv')

In [1827]:
loan_df.head()
loan_df['loan_status']

0        Charged Off
1         Fully Paid
2         Fully Paid
3            Current
4         Fully Paid
            ...     
24994     Fully Paid
24995     Fully Paid
24996     Fully Paid
24997     Fully Paid
24998     Fully Paid
Name: loan_status, Length: 24999, dtype: object

In [1828]:
loan_df.head(10)

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,573354,737474,10000,10000,9950.000000,36 months,7.51%,311.11,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,476321,603324,15000,15000,14800.000000,36 months,8.94%,476.58,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,451484,556265,2000,2000,2000.000000,36 months,13.57%,67.94,C,C3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1018129,1246557,35000,35000,33951.844130,60 months,20.89%,944.71,F,F1,...,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN
4,800018,1005270,14000,14000,14000.000000,60 months,17.49%,351.64,D,D5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
5,471391,595223,12000,12000,11900.000000,36 months,8.94%,381.26,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
6,521762,674784,10000,6525,6480.473624,60 months,7.88%,131.93,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
7,664532,849696,7800,7800,7800.000000,60 months,14.91%,185.20,D,D2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
8,973622,1195649,18825,18825,18825.000000,36 months,7.90%,589.04,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
9,882849,1098094,35000,26675,26181.232420,60 months,11.71%,589.47,B,B3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN


Seperated labels column and converted categorical datatype into numerical

In [1829]:
loan_df["loan_status"].replace({"Fully Paid": +1, "Charged Off": -1,'Current':0}, inplace=True)
loan_df["loan_status"]

0       -1
1        1
2        1
3        0
4        1
        ..
24994    1
24995    1
24996    1
24997    1
24998    1
Name: loan_status, Length: 24999, dtype: int64

In [1830]:
train_data=loan_df.drop(columns='loan_status')
train_data
train_labels=loan_df['loan_status']
train_labels

0       -1
1        1
2        1
3        0
4        1
        ..
24994    1
24995    1
24996    1
24997    1
24998    1
Name: loan_status, Length: 24999, dtype: int64

Preprocessed input data attributes 

In [1831]:
train_data.dtypes

id                              int64
member_id                       int64
loan_amnt                       int64
funded_amnt                     int64
funded_amnt_inv               float64
                               ...   
tax_liens                     float64
tot_hi_cred_lim               float64
total_bal_ex_mort             float64
total_bc_limit                float64
total_il_high_credit_limit    float64
Length: 110, dtype: object

In [1832]:
dfp = train_data['int_rate'].str.rstrip('%').astype(float) / 100   #Converted interest column percentages into float values 
train_data=train_data.assign(int_rate=dfp)
train_data

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,573354,737474,10000,10000,9950.00000,36 months,0.0751,311.11,A,A4,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,476321,603324,15000,15000,14800.00000,36 months,0.0894,476.58,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,451484,556265,2000,2000,2000.00000,36 months,0.1357,67.94,C,C3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1018129,1246557,35000,35000,33951.84413,60 months,0.2089,944.71,F,F1,...,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN
4,800018,1005270,14000,14000,14000.00000,60 months,0.1749,351.64,D,D5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,450579,554340,5500,5500,5500.00000,36 months,0.1496,190.55,D,D2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
24995,788033,991661,11450,11450,11450.00000,36 months,0.0849,361.40,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
24996,568459,731299,24000,24000,21100.31424,60 months,0.2053,642.96,G,G2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
24997,397594,428786,14000,14000,11411.12089,36 months,0.1758,503.19,F,F2,...,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN


In [1833]:
train_data_obj=train_data.select_dtypes(include=['object']).copy   #no.of columns are of object datatype
train_data_obj

<bound method NDFrame.copy of             term grade sub_grade            emp_title emp_length  \
0      36 months     A        A4                   15  10+ years   
1      36 months     A        A5                 1400     1 year   
2      36 months     C        C3                36000    4 years   
3      60 months     F        F1            553742017  10+ years   
4      60 months     D        D5         old palm inc    5 years   
...          ...   ...       ...                  ...        ...   
24994  36 months     D        D2                 PPDG    7 years   
24995  36 months     A        A5       PPG Industries    7 years   
24996  60 months     G        G2       PPG Industries  10+ years   
24997  36 months     F        F2       PPG Industries  10+ years   
24998  36 months     C        C2  PPG Industries, INC   < 1 year   

      home_ownership verification_status issue_d pymnt_plan  \
0               RENT     Source Verified  10-Sep          n   
1               RENT       

In [1834]:
train_data["term"].replace({"36 months": 36, "60 months": 60}, inplace=True) #Took term as numerical values
train_data["term"]

0        36
1        36
2        36
3        60
4        60
         ..
24994    36
24995    36
24996    60
24997    36
24998    36
Name: term, Length: 24999, dtype: int64

In [1835]:
train_data.drop('grade',inplace=True,axis=1)     #Removed Grade column
train_data.drop('sub_grade',inplace=True,axis=1)  #Removed sub_grade column

In [1836]:
train_data['acc_now_delinq'].mean() 

0.0

In [1837]:


train_data.drop('acc_now_delinq',inplace=True,axis=1)  #Dropped as the column mean is zero 

In [1838]:
train_data['chargeoff_within_12_mths'].mean()

0.0

In [1839]:
train_data.drop('chargeoff_within_12_mths',inplace=True,axis=1)  #Dropped as the column mean is zero 

In [1840]:
train_data


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,573354,737474,10000,10000,9950.00000,36,0.0751,311.11,15,10+ years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,476321,603324,15000,15000,14800.00000,36,0.0894,476.58,1400,1 year,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,451484,556265,2000,2000,2000.00000,36,0.1357,67.94,36000,4 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,1018129,1246557,35000,35000,33951.84413,60,0.2089,944.71,553742017,10+ years,...,NaN,NaN,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN
4,800018,1005270,14000,14000,14000.00000,60,0.1749,351.64,old palm inc,5 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,450579,554340,5500,5500,5500.00000,36,0.1496,190.55,PPDG,7 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
24995,788033,991661,11450,11450,11450.00000,36,0.0849,361.40,PPG Industries,7 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
24996,568459,731299,24000,24000,21100.31424,60,0.2053,642.96,PPG Industries,10+ years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
24997,397594,428786,14000,14000,11411.12089,36,0.1758,503.19,PPG Industries,10+ years,...,NaN,NaN,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN


In [1841]:
train_data = train_data.loc[:, (train_data != 0).any(axis=0)]  #Removed those columns which has all 0 values

In [1842]:
df=train_data  #assigned train_data as df

In [1843]:
df=df.dropna(axis=1, how='all')  #removed all those columns which are filled with NaN,Nas


In [1844]:
df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,...,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,pub_rec_bankruptcies,tax_liens
0,573354,737474,10000,10000,9950.00000,36,0.0751,311.11,15,10+ years,...,3.96,11-Mar,311.11,NaN,11-Jul,0.0,1,INDIVIDUAL,0.0,0.0
1,476321,603324,15000,15000,14800.00000,36,0.0894,476.58,1400,1 year,...,0.00,10-Mar,15114.03,NaN,10-Feb,0.0,1,INDIVIDUAL,0.0,0.0
2,451484,556265,2000,2000,2000.00000,36,0.1357,67.94,36000,4 years,...,0.00,11-Dec,101.78,NaN,11-Dec,0.0,1,INDIVIDUAL,0.0,0.0
3,1018129,1246557,35000,35000,33951.84413,60,0.2089,944.71,553742017,10+ years,...,0.00,16-May,944.71,16-Jun,16-May,0.0,1,INDIVIDUAL,2.0,0.0
4,800018,1005270,14000,14000,14000.00000,60,0.1749,351.64,old palm inc,5 years,...,0.00,15-Sep,3943.27,NaN,16-May,0.0,1,INDIVIDUAL,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,450579,554340,5500,5500,5500.00000,36,0.1496,190.55,PPDG,7 years,...,0.00,12-May,1122.20,NaN,12-Jun,0.0,1,INDIVIDUAL,0.0,0.0
24995,788033,991661,11450,11450,11450.00000,36,0.0849,361.40,PPG Industries,7 years,...,0.00,14-Jul,382.90,NaN,16-May,0.0,1,INDIVIDUAL,0.0,0.0
24996,568459,731299,24000,24000,21100.31424,60,0.2053,642.96,PPG Industries,10+ years,...,0.00,13-Jan,16500.39,NaN,16-May,0.0,1,INDIVIDUAL,0.0,0.0
24997,397594,428786,14000,14000,11411.12089,36,0.1758,503.19,PPG Industries,10+ years,...,0.00,11-Oct,3857.88,NaN,15-Jun,0.0,1,INDIVIDUAL,1.0,0.0


Started eye-balling each object datatype column and decided to continue with data or not

In [1845]:
df['next_pymnt_d']



0           NaN
1           NaN
2           NaN
3        16-Jun
4           NaN
          ...  
24994       NaN
24995       NaN
24996       NaN
24997       NaN
24998       NaN
Name: next_pymnt_d, Length: 24999, dtype: object

In [1846]:
df.drop('next_pymnt_d',inplace=True,axis=1)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [1847]:
df

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,...,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,pub_rec_bankruptcies,tax_liens
0,573354,737474,10000,10000,9950.00000,36,0.0751,311.11,15,10+ years,...,386.43,3.96,11-Mar,311.11,11-Jul,0.0,1,INDIVIDUAL,0.0,0.0
1,476321,603324,15000,15000,14800.00000,36,0.0894,476.58,1400,1 year,...,0.00,0.00,10-Mar,15114.03,10-Feb,0.0,1,INDIVIDUAL,0.0,0.0
2,451484,556265,2000,2000,2000.00000,36,0.1357,67.94,36000,4 years,...,0.00,0.00,11-Dec,101.78,11-Dec,0.0,1,INDIVIDUAL,0.0,0.0
3,1018129,1246557,35000,35000,33951.84413,60,0.2089,944.71,553742017,10+ years,...,0.00,0.00,16-May,944.71,16-May,0.0,1,INDIVIDUAL,2.0,0.0
4,800018,1005270,14000,14000,14000.00000,60,0.1749,351.64,old palm inc,5 years,...,0.00,0.00,15-Sep,3943.27,16-May,0.0,1,INDIVIDUAL,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,450579,554340,5500,5500,5500.00000,36,0.1496,190.55,PPDG,7 years,...,0.00,0.00,12-May,1122.20,12-Jun,0.0,1,INDIVIDUAL,0.0,0.0
24995,788033,991661,11450,11450,11450.00000,36,0.0849,361.40,PPG Industries,7 years,...,0.00,0.00,14-Jul,382.90,16-May,0.0,1,INDIVIDUAL,0.0,0.0
24996,568459,731299,24000,24000,21100.31424,60,0.2053,642.96,PPG Industries,10+ years,...,0.00,0.00,13-Jan,16500.39,16-May,0.0,1,INDIVIDUAL,0.0,0.0
24997,397594,428786,14000,14000,11411.12089,36,0.1758,503.19,PPG Industries,10+ years,...,0.00,0.00,11-Oct,3857.88,15-Jun,0.0,1,INDIVIDUAL,1.0,0.0


In [1848]:
df.application_type.unique()  #have seen no.of unique values per column and decided

array(['INDIVIDUAL'], dtype=object)

In [1849]:
df['pymnt_plan']

0        n
1        n
2        n
3        n
4        n
        ..
24994    n
24995    n
24996    n
24997    n
24998    n
Name: pymnt_plan, Length: 24999, dtype: object

In [1850]:
df.drop('pymnt_plan',inplace=True,axis=1) #It contains only one type

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [1851]:
df.drop('issue_d',inplace=True,axis=1)

In [1852]:
df.drop('url',inplace=True,axis=1)

In [1853]:
df.drop('desc',inplace=True,axis=1)

In [1854]:
df.drop('title',inplace=True,axis=1)

In [1855]:
df.drop('zip_code',inplace=True,axis=1)

In [1856]:
df.drop('addr_state',inplace=True,axis=1)

In [1857]:
df.drop('initial_list_status',inplace=True,axis=1) #It contains only one type

In [1858]:
df.drop('collections_12_mths_ex_med',inplace=True,axis=1) #It contains only 0s and Nan type

In [1859]:
df.drop('tax_liens',inplace=True,axis=1) #It contains only 0s and Nan type

In [1860]:
df.drop('application_type',inplace=True,axis=1)

In [1861]:
dfp1 = df['revol_util'].str.rstrip('%').astype(float) / 100
df=df.assign(int_rate=dfp1)


In [1862]:
df.drop('last_pymnt_d',inplace=True,axis=1)

In [1863]:
df.drop('policy_code',inplace=True,axis=1)

In [1864]:
df.drop('id',inplace=True,axis=1)

In [1865]:
df.drop('member_id',inplace=True,axis=1)

In [1866]:
df


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,...,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,10000,10000,9950.00000,36,0.012,311.11,15,10+ years,RENT,30000.0,...,2247.210000,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,11-Jul,0.0
1,15000,15000,14800.00000,36,0.140,476.58,1400,1 year,RENT,147000.0,...,15112.760000,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,10-Feb,0.0
2,2000,2000,2000.00000,36,0.344,67.94,36000,4 years,OWN,36000.0,...,2354.966827,2354.97,2000.00,354.97,0.0,0.00,0.00,101.78,11-Dec,0.0
3,35000,35000,33951.84413,60,0.914,944.71,553742017,10+ years,MORTGAGE,160000.0,...,50974.920000,48442.03,29644.65,21330.27,0.0,0.00,0.00,944.71,16-May,2.0
4,14000,14000,14000.00000,60,0.094,351.64,old palm inc,5 years,MORTGAGE,50000.0,...,20804.230020,20804.23,14000.00,6804.23,0.0,0.00,0.00,3943.27,16-May,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,5500,5500,5500.00000,36,0.582,190.55,PPDG,7 years,MORTGAGE,52000.0,...,6825.066783,6825.07,5500.00,1325.07,0.0,0.00,0.00,1122.20,12-Jun,0.0
24995,11450,11450,11450.00000,36,0.340,361.40,PPG Industries,7 years,RENT,40000.0,...,13010.206030,13010.21,11450.00,1560.21,0.0,0.00,0.00,382.90,16-May,0.0
24996,24000,24000,21100.31424,60,0.772,642.96,PPG Industries,10+ years,MORTGAGE,74454.0,...,33765.068310,26978.96,24000.00,9765.07,0.0,0.00,0.00,16500.39,16-May,0.0
24997,14000,14000,11411.12089,36,0.950,503.19,PPG Industries,10+ years,MORTGAGE,86000.0,...,17917.012900,14191.06,14000.00,3917.01,0.0,0.00,0.00,3857.88,15-Jun,1.0


In [1867]:
df.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'emp_title', 'emp_length', 'home_ownership',
       'annual_inc', 'verification_status', 'purpose', 'dti', 'delinq_2yrs',
       'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq',
       'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal',
       'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_amnt', 'last_credit_pull_d', 'pub_rec_bankruptcies'],
      dtype='object')

In [1868]:
df_obj=df.select_dtypes(include=['object']).copy
df_obj

<bound method NDFrame.copy of                  emp_title emp_length home_ownership verification_status  \
0                       15  10+ years           RENT     Source Verified   
1                     1400     1 year           RENT        Not Verified   
2                    36000    4 years            OWN        Not Verified   
3                553742017  10+ years       MORTGAGE            Verified   
4             old palm inc    5 years       MORTGAGE        Not Verified   
...                    ...        ...            ...                 ...   
24994                 PPDG    7 years       MORTGAGE        Not Verified   
24995       PPG Industries    7 years           RENT     Source Verified   
24996       PPG Industries  10+ years       MORTGAGE            Verified   
24997       PPG Industries  10+ years       MORTGAGE            Verified   
24998  PPG Industries, INC   < 1 year           RENT        Not Verified   

                  purpose earliest_cr_line revol_util las

In [1869]:
len(df['emp_title'].unique())  #It has 19820 unique values so removed 

19820

In [1870]:
df.drop('emp_title',inplace=True,axis=1)

In [1871]:
(df['emp_length'].unique())

array(['10+ years', '1 year', '4 years', '5 years', '7 years', '6 years',
       '2 years', '8 years', '< 1 year', '3 years', '9 years', nan],
      dtype=object)

In [1872]:
df["emp_length"].replace({"1 year": +1, "4 years": 4,'5 years':5,'7 years':7,'6 years':6,'2 years':6,'8 years':8,'3 years':3,'9 years':9,'< 1 year':0,'10+ years':10,'nan':0}, inplace=True)

In [1873]:
df['emp_length'] #Changed emp_lenth column in numeric data type

0        10.0
1         1.0
2         4.0
3        10.0
4         5.0
         ... 
24994     7.0
24995     7.0
24996    10.0
24997    10.0
24998     0.0
Name: emp_length, Length: 24999, dtype: float64

In [1874]:
df['home_ownership'].unique()

array(['RENT', 'OWN', 'MORTGAGE', 'OTHER'], dtype=object)

In [1875]:
df['home_ownership'].replace({'RENT':1,'OWN':2,"MORTGAGE":-2,"OTHER":0},inplace=True) #Changed home_ownership column in numeric data type

In [1876]:
df['verification_status'].unique()

array(['Source Verified', 'Not Verified', 'Verified'], dtype=object)

In [1877]:
df['verification_status'].replace({"Source Verified":0,'Not Verified':-1,'Verified':+1},inplace=True)

In [1878]:
(df['purpose'].unique())

array(['home_improvement', 'other', 'major_purchase',
       'debt_consolidation', 'credit_card', 'house', 'small_business',
       'wedding', 'medical', 'moving', 'car', 'educational', 'vacation',
       'renewable_energy'], dtype=object)

In [1879]:
df['purpose'].replace({'home_improvement':2,'other':1,'major_purchase':3,'debt_consolidation':0,'credit_card':0,'house':4,'small_business':3,'wedding':1,'medical':3,'moving':2,'car':3,'educational':4,'vacation':1,'renewable_energy':3},inplace=True)

In [1880]:
df.drop('earliest_cr_line',inplace=True,axis=1)  #Removed Dates

In [1881]:
df.drop('last_credit_pull_d',inplace=True,axis=1)
df.drop('funded_amnt',inplace=True,axis=1)
df.drop('funded_amnt_inv',inplace=True,axis=1)

In [1882]:
dfp4 = df['revol_util'].str.rstrip('%').astype(float) / 100   #Conveted revol_util column percentages into float values 
df=df.assign(revol_util=dfp4)
df

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,10000,36,0.012,311.11,10.0,1,30000.0,0,2,5.00,...,0.0,2247.210000,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,0.0
1,15000,36,0.140,476.58,1.0,1,147000.0,-1,1,3.47,...,0.0,15112.760000,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,0.0
2,2000,36,0.344,67.94,4.0,2,36000.0,-1,3,7.83,...,0.0,2354.966827,2354.97,2000.00,354.97,0.0,0.00,0.00,101.78,0.0
3,35000,60,0.914,944.71,10.0,-2,160000.0,1,0,12.11,...,5340.0,50974.920000,48442.03,29644.65,21330.27,0.0,0.00,0.00,944.71,2.0
4,14000,60,0.094,351.64,5.0,-2,50000.0,-1,0,21.24,...,0.0,20804.230020,20804.23,14000.00,6804.23,0.0,0.00,0.00,3943.27,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,5500,36,0.582,190.55,7.0,-2,52000.0,-1,0,18.74,...,0.0,6825.066783,6825.07,5500.00,1325.07,0.0,0.00,0.00,1122.20,0.0
24995,11450,36,0.340,361.40,7.0,1,40000.0,0,0,21.21,...,0.0,13010.206030,13010.21,11450.00,1560.21,0.0,0.00,0.00,382.90,0.0
24996,24000,60,0.772,642.96,10.0,-2,74454.0,1,0,19.95,...,0.0,33765.068310,26978.96,24000.00,9765.07,0.0,0.00,0.00,16500.39,0.0
24997,14000,36,0.950,503.19,10.0,-2,86000.0,1,0,21.93,...,0.0,17917.012900,14191.06,14000.00,3917.01,0.0,0.00,0.00,3857.88,1.0


In [1883]:
df

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,10000,36,0.012,311.11,10.0,1,30000.0,0,2,5.00,...,0.0,2247.210000,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,0.0
1,15000,36,0.140,476.58,1.0,1,147000.0,-1,1,3.47,...,0.0,15112.760000,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,0.0
2,2000,36,0.344,67.94,4.0,2,36000.0,-1,3,7.83,...,0.0,2354.966827,2354.97,2000.00,354.97,0.0,0.00,0.00,101.78,0.0
3,35000,60,0.914,944.71,10.0,-2,160000.0,1,0,12.11,...,5340.0,50974.920000,48442.03,29644.65,21330.27,0.0,0.00,0.00,944.71,2.0
4,14000,60,0.094,351.64,5.0,-2,50000.0,-1,0,21.24,...,0.0,20804.230020,20804.23,14000.00,6804.23,0.0,0.00,0.00,3943.27,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,5500,36,0.582,190.55,7.0,-2,52000.0,-1,0,18.74,...,0.0,6825.066783,6825.07,5500.00,1325.07,0.0,0.00,0.00,1122.20,0.0
24995,11450,36,0.340,361.40,7.0,1,40000.0,0,0,21.21,...,0.0,13010.206030,13010.21,11450.00,1560.21,0.0,0.00,0.00,382.90,0.0
24996,24000,60,0.772,642.96,10.0,-2,74454.0,1,0,19.95,...,0.0,33765.068310,26978.96,24000.00,9765.07,0.0,0.00,0.00,16500.39,0.0
24997,14000,36,0.950,503.19,10.0,-2,86000.0,1,0,21.93,...,0.0,17917.012900,14191.06,14000.00,3917.01,0.0,0.00,0.00,3857.88,1.0


All cells of object data type are converted to numeric

In [1884]:
df_obj=df.select_dtypes(include=['object']).copy
df_obj

<bound method NDFrame.copy of Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[24999 rows x 0 columns]>

In [1885]:
df.fillna(method ='pad') #Filled all null values with previous one

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,10000,36,0.012,311.11,10.0,1,30000.0,0,2,5.00,...,0.0,2247.210000,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,0.0
1,15000,36,0.140,476.58,1.0,1,147000.0,-1,1,3.47,...,0.0,15112.760000,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,0.0
2,2000,36,0.344,67.94,4.0,2,36000.0,-1,3,7.83,...,0.0,2354.966827,2354.97,2000.00,354.97,0.0,0.00,0.00,101.78,0.0
3,35000,60,0.914,944.71,10.0,-2,160000.0,1,0,12.11,...,5340.0,50974.920000,48442.03,29644.65,21330.27,0.0,0.00,0.00,944.71,2.0
4,14000,60,0.094,351.64,5.0,-2,50000.0,-1,0,21.24,...,0.0,20804.230020,20804.23,14000.00,6804.23,0.0,0.00,0.00,3943.27,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,5500,36,0.582,190.55,7.0,-2,52000.0,-1,0,18.74,...,0.0,6825.066783,6825.07,5500.00,1325.07,0.0,0.00,0.00,1122.20,0.0
24995,11450,36,0.340,361.40,7.0,1,40000.0,0,0,21.21,...,0.0,13010.206030,13010.21,11450.00,1560.21,0.0,0.00,0.00,382.90,0.0
24996,24000,60,0.772,642.96,10.0,-2,74454.0,1,0,19.95,...,0.0,33765.068310,26978.96,24000.00,9765.07,0.0,0.00,0.00,16500.39,0.0
24997,14000,36,0.950,503.19,10.0,-2,86000.0,1,0,21.93,...,0.0,17917.012900,14191.06,14000.00,3917.01,0.0,0.00,0.00,3857.88,1.0


In [1886]:
df.isnull().sum() #checking columns which has null 

loan_amnt                      0
term                           0
int_rate                      29
installment                    0
emp_length                    41
home_ownership                 0
annual_inc                     0
verification_status            0
purpose                        0
dti                            0
delinq_2yrs                    0
inq_last_6mths                 0
mths_since_last_delinq     16240
mths_since_last_record     23320
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                    29
total_acc                      0
out_prncp                      0
out_prncp_inv                  0
total_pymnt                    0
total_pymnt_inv                0
total_rec_prncp                0
total_rec_int                  0
total_rec_late_fee             0
recoveries                     0
collection_recovery_fee        0
last_pymnt_amnt                0
pub_rec_bankruptcies         417
dtype: int

In [1887]:
df.fillna(df.mean())

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,10000,36,0.012,311.11,10.0,1,30000.0,0,2,5.00,...,0.0,2247.210000,2235.98,1509.91,350.87,0.0,386.43,3.96,311.11,0.0
1,15000,36,0.140,476.58,1.0,1,147000.0,-1,1,3.47,...,0.0,15112.760000,14911.26,15000.00,112.76,0.0,0.00,0.00,15114.03,0.0
2,2000,36,0.344,67.94,4.0,2,36000.0,-1,3,7.83,...,0.0,2354.966827,2354.97,2000.00,354.97,0.0,0.00,0.00,101.78,0.0
3,35000,60,0.914,944.71,10.0,-2,160000.0,1,0,12.11,...,5340.0,50974.920000,48442.03,29644.65,21330.27,0.0,0.00,0.00,944.71,2.0
4,14000,60,0.094,351.64,5.0,-2,50000.0,-1,0,21.24,...,0.0,20804.230020,20804.23,14000.00,6804.23,0.0,0.00,0.00,3943.27,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24994,5500,36,0.582,190.55,7.0,-2,52000.0,-1,0,18.74,...,0.0,6825.066783,6825.07,5500.00,1325.07,0.0,0.00,0.00,1122.20,0.0
24995,11450,36,0.340,361.40,7.0,1,40000.0,0,0,21.21,...,0.0,13010.206030,13010.21,11450.00,1560.21,0.0,0.00,0.00,382.90,0.0
24996,24000,60,0.772,642.96,10.0,-2,74454.0,1,0,19.95,...,0.0,33765.068310,26978.96,24000.00,9765.07,0.0,0.00,0.00,16500.39,0.0
24997,14000,36,0.950,503.19,10.0,-2,86000.0,1,0,21.93,...,0.0,17917.012900,14191.06,14000.00,3917.01,0.0,0.00,0.00,3857.88,1.0


In [1888]:
df.isnull().sum() #checking columns which has null 

loan_amnt                      0
term                           0
int_rate                      29
installment                    0
emp_length                    41
home_ownership                 0
annual_inc                     0
verification_status            0
purpose                        0
dti                            0
delinq_2yrs                    0
inq_last_6mths                 0
mths_since_last_delinq     16240
mths_since_last_record     23320
open_acc                       0
pub_rec                        0
revol_bal                      0
revol_util                    29
total_acc                      0
out_prncp                      0
out_prncp_inv                  0
total_pymnt                    0
total_pymnt_inv                0
total_rec_prncp                0
total_rec_int                  0
total_rec_late_fee             0
recoveries                     0
collection_recovery_fee        0
last_pymnt_amnt                0
pub_rec_bankruptcies         417
dtype: int

In [1889]:
df['pub_rec_bankruptcies'].fillna(int(df['pub_rec_bankruptcies'].mean()), inplace=True)
df['int_rate'].fillna(int(df['int_rate'].mean()), inplace=True)
df['emp_length'].fillna(int(df['emp_length'].mean()), inplace=True)
df['mths_since_last_delinq'].fillna(int(df['mths_since_last_delinq'].mean()), inplace=True)
df['mths_since_last_record'].fillna(int(df['mths_since_last_record'].mean()), inplace=True)
df['revol_util'].fillna(int(df['revol_util'].mean()), inplace=True)


In [1890]:
df.isnull().sum() #All nulls are assigned with their mean values of respective columns
df.drop('revol_bal',inplace=True,axis=1)



df.drop('total_rec_late_fee',inplace=True,axis=1)


In [1891]:
df.shape

(24999, 28)

Followed same steps with test data

In [1892]:
test_data=pd.read_csv("loan_test.csv")

In [1893]:
dft=test_data

In [1894]:
dft['loan_status']

0         Fully Paid
1        Charged Off
2         Fully Paid
3         Fully Paid
4         Fully Paid
            ...     
14713     Fully Paid
14714     Fully Paid
14715     Fully Paid
14716     Fully Paid
14717     Fully Paid
Name: loan_status, Length: 14718, dtype: object

In [1895]:
dft["loan_status"].replace({"Fully Paid": +1, "Charged Off": -1,'Current':0}, inplace=True)
dft["loan_status"]

0        1
1       -1
2        1
3        1
4        1
        ..
14713    1
14714    1
14715    1
14716    1
14717    1
Name: loan_status, Length: 14718, dtype: int64

In [1896]:
df1=dft.drop(columns='loan_status')
df1
test_labels=dft['loan_status']
test_labels

0        1
1       -1
2        1
3        1
4        1
        ..
14713    1
14714    1
14715    1
14716    1
14717    1
Name: loan_status, Length: 14718, dtype: int64

In [1897]:
df1

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1001205,1227186,4200,4200,4200.0,36 months,6.62%,128.96,A,A2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,850106,1061893,1325,1325,1325.0,36 months,13.49%,44.96,C,C2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,444243,541365,10000,10000,9975.0,36 months,8.94%,317.72,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,741007,938805,6000,6000,6000.0,60 months,16.49%,147.48,D,D3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,572468,736352,25000,25000,25000.0,36 months,15.21%,869.21,D,D2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,98276,98268,5400,5400,200.0,36 months,7.75%,168.60,A,A3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14714,93277,93254,3000,3000,950.0,36 months,8.70%,94.98,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14715,92676,92671,5000,5000,150.0,36 months,8.07%,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14716,90395,90390,5000,5000,1325.0,36 months,8.07%,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1898]:
dfp5 = df1['int_rate'].str.rstrip('%').astype(float) / 100   #Conveted interest column percentages into float values 
df1=df1.assign(int_rate=dfp)
df1["term"].replace({"36 months": 36, "60 months": 60}, inplace=True) #Took term as numerical valus
df1


,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1001205,1227186,4200,4200,4200.0,36,0.0751,128.96,A,A2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,850106,1061893,1325,1325,1325.0,36,0.0894,44.96,C,C2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,444243,541365,10000,10000,9975.0,36,0.1357,317.72,A,A5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,741007,938805,6000,6000,6000.0,60,0.2089,147.48,D,D3,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,572468,736352,25000,25000,25000.0,36,0.1749,869.21,D,D2,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,98276,98268,5400,5400,200.0,36,0.0599,168.60,A,A3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14714,93277,93254,3000,3000,950.0,36,0.0790,94.98,B,B1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14715,92676,92671,5000,5000,150.0,36,0.1682,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14716,90395,90390,5000,5000,1325.0,36,0.1411,156.84,A,A4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1899]:
df1.drop('grade',inplace=True,axis=1)     #Removed Grade column
df1.drop('sub_grade',inplace=True,axis=1)  #Removed sub_grade column
df1.drop('acc_now_delinq',inplace=True,axis=1)  #Dropped as the column mean is zero 
df1.drop('chargeoff_within_12_mths',inplace=True,axis=1)  #Dropped as the column mean is zero 
df1 = df1.loc[:, (df1 != 0).any(axis=0)]  #Removed those columns which has all 0 values
df1

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,...,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit
0,1001205,1227186,4200,4200,4200.0,36,0.0751,128.96,"PPG Industries, Inc.",6 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
1,850106,1061893,1325,1325,1325.0,36,0.0894,44.96,PPI Technology Services,< 1 year,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2,444243,541365,10000,10000,9975.0,36,0.1357,317.72,PPL Corporation,4 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3,741007,938805,6000,6000,6000.0,60,0.2089,147.48,ppl eu,10+ years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,572468,736352,25000,25000,25000.0,36,0.1749,869.21,PPMM,4 years,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,98276,98268,5400,5400,200.0,36,0.0599,168.60,NaN,< 1 year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14714,93277,93254,3000,3000,950.0,36,0.0790,94.98,NaN,< 1 year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14715,92676,92671,5000,5000,150.0,36,0.1682,156.84,NaN,< 1 year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14716,90395,90390,5000,5000,1325.0,36,0.1411,156.84,NaN,< 1 year,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1900]:
df1=df1.dropna(axis=1, how='all')
df1.drop('next_pymnt_d',inplace=True,axis=1)
df1.drop('pymnt_plan',inplace=True,axis=1) #It contains only one type
df1.drop('issue_d',inplace=True,axis=1)
df1.drop('url',inplace=True,axis=1)
df1.drop('desc',inplace=True,axis=1)
df1.drop('title',inplace=True,axis=1)
df1.drop('zip_code',inplace=True,axis=1)
df1.drop('addr_state',inplace=True,axis=1)
df1.drop('initial_list_status',inplace=True,axis=1) #It contains only one type
df1.drop('collections_12_mths_ex_med',inplace=True,axis=1) #It contains only 0s and Nan type
df1.drop('tax_liens',inplace=True,axis=1) #It contains only 0s and Nan type
df1.drop('application_type',inplace=True,axis=1)
dfp7 = df1['revol_util'].str.rstrip('%').astype(float) / 100
df1=df1.assign(int_rate=dfp7)
df1.drop('last_pymnt_d',inplace=True,axis=1)
df1.drop('policy_code',inplace=True,axis=1)
df1.drop('id',inplace=True,axis=1)
df1.drop('member_id',inplace=True,axis=1)
df1.drop('revol_bal',inplace=True,axis=1)
# df1.drop('home_ownership',inplace=True,axis=1)
# df1.drop('revol_util',inplace=True,axis=1)


df1.drop('total_rec_late_fee',inplace=True,axis=1)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [1901]:
df1

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_title,emp_length,home_ownership,annual_inc,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,4200,4200,4200.0,36,0.116,128.96,"PPG Industries, Inc.",6 years,RENT,69000.0,...,0.0,4642.374867,4642.37,4200.00,442.37,0.0,0.000,132.82,16-May,0.0
1,1325,1325,1325.0,36,0.119,44.96,PPI Technology Services,< 1 year,RENT,24000.0,...,0.0,1476.640000,1476.64,1145.61,287.60,28.5,0.285,44.96,14-Sep,0.0
2,10000,10000,9975.0,36,0.464,317.72,PPL Corporation,4 years,MORTGAGE,75000.0,...,0.0,10287.347790,10261.63,10000.00,287.35,0.0,0.000,9337.10,16-May,0.0
3,6000,6000,6000.0,60,0.602,147.48,ppl eu,10+ years,RENT,60000.0,...,0.0,8848.343924,8848.34,6000.00,2848.34,0.0,0.000,147.02,16-May,0.0
4,25000,25000,25000.0,36,0.557,869.21,PPMM,4 years,RENT,135000.0,...,0.0,31293.587380,31293.59,25000.00,6293.59,0.0,0.000,960.31,16-May,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,5400,5400,200.0,36,0.165,168.60,NaN,< 1 year,RENT,8000.0,...,0.0,6069.341452,224.80,5400.00,669.34,0.0,0.000,169.09,13-Dec,NaN
14714,3000,3000,950.0,36,0.078,94.98,NaN,< 1 year,MORTGAGE,25000.0,...,0.0,3419.144850,1082.73,3000.00,419.14,0.0,0.000,97.45,10-Jul,NaN
14715,5000,5000,150.0,36,0.263,156.84,NaN,< 1 year,MORTGAGE,180000.0,...,0.0,5645.957239,169.38,5000.00,645.96,0.0,0.000,157.31,7-Jun,NaN
14716,5000,5000,1325.0,36,0.194,156.84,NaN,< 1 year,MORTGAGE,100000.0,...,0.0,5272.161128,1397.12,5000.00,272.16,0.0,0.000,0.00,7-Jun,NaN


In [1902]:
df1_obj=df1.select_dtypes(include=['object']).copy
df1_obj

<bound method NDFrame.copy of                      emp_title emp_length home_ownership verification_status  \
0         PPG Industries, Inc.    6 years           RENT        Not Verified   
1      PPI Technology Services   < 1 year           RENT     Source Verified   
2              PPL Corporation    4 years       MORTGAGE        Not Verified   
3                       ppl eu  10+ years           RENT     Source Verified   
4                         PPMM    4 years           RENT     Source Verified   
...                        ...        ...            ...                 ...   
14713                      NaN   < 1 year           RENT        Not Verified   
14714                      NaN   < 1 year       MORTGAGE        Not Verified   
14715                      NaN   < 1 year       MORTGAGE        Not Verified   
14716                      NaN   < 1 year       MORTGAGE        Not Verified   
14717                      NaN   < 1 year       MORTGAGE        Not Verified   

         

In [1903]:
df1.drop('emp_title',inplace=True,axis=1)
df1.drop('funded_amnt',inplace=True,axis=1)
df1.drop('funded_amnt_inv',inplace=True,axis=1)

df1["emp_length"].replace({"1 year": +1, "4 years": 4,'5 years':5,'7 years':7,'6 years':6,'2 years':6,'8 years':8,'3 years':3,'9 years':9,'< 1 year':0,'10+ years':10,'nan':0}, inplace=True)
df1['emp_length']

0         6.0
1         0.0
2         4.0
3        10.0
4         4.0
         ... 
14713     0.0
14714     0.0
14715     0.0
14716     0.0
14717     0.0
Name: emp_length, Length: 14718, dtype: float64

In [1904]:
df1['home_ownership'].replace({'RENT':1,'OWN':2,"MORTGAGE":-2,"OTHER":0,"NONE":0},inplace=True) #Changed home_ownership column in numeric data type
df1['verification_status'].replace({"Source Verified":0,'Not Verified':-1,'Verified':+1},inplace=True)
df1['purpose'].replace({'home_improvement':2,'other':1,'major_purchase':3,'debt_consolidation':0,'credit_card':0,'house':4,'small_business':3,'wedding':1,'medical':3,'moving':2,'car':3,'educational':4,'vacation':1,'renewable_energy':3},inplace=True)
df1.drop('earliest_cr_line',inplace=True,axis=1)  #Removed Dates
df1.drop('last_credit_pull_d',inplace=True,axis=1)
dfp8 = df1['revol_util'].str.rstrip('%').astype(float) / 100   #Conveted revol_util column percentages into float values 
df1=df1.assign(revol_util=dfp8)
df1

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,4200,36,0.116,128.96,6.0,1,69000.0,-1,3,11.08,...,0.0,0.0,4642.374867,4642.37,4200.00,442.37,0.0,0.000,132.82,0.0
1,1325,36,0.119,44.96,0.0,1,24000.0,0,2,3.80,...,0.0,0.0,1476.640000,1476.64,1145.61,287.60,28.5,0.285,44.96,0.0
2,10000,36,0.464,317.72,4.0,-2,75000.0,-1,0,12.64,...,0.0,0.0,10287.347790,10261.63,10000.00,287.35,0.0,0.000,9337.10,0.0
3,6000,60,0.602,147.48,10.0,1,60000.0,0,0,13.06,...,0.0,0.0,8848.343924,8848.34,6000.00,2848.34,0.0,0.000,147.02,0.0
4,25000,36,0.557,869.21,4.0,1,135000.0,0,0,15.49,...,0.0,0.0,31293.587380,31293.59,25000.00,6293.59,0.0,0.000,960.31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,5400,36,0.165,168.60,0.0,1,8000.0,-1,4,3.00,...,0.0,0.0,6069.341452,224.80,5400.00,669.34,0.0,0.000,169.09,NaN
14714,3000,36,0.078,94.98,0.0,-2,25000.0,-1,3,14.54,...,0.0,0.0,3419.144850,1082.73,3000.00,419.14,0.0,0.000,97.45,NaN
14715,5000,36,0.263,156.84,0.0,-2,180000.0,-1,2,5.55,...,0.0,0.0,5645.957239,169.38,5000.00,645.96,0.0,0.000,157.31,NaN
14716,5000,36,0.194,156.84,0.0,-2,100000.0,-1,0,2.30,...,0.0,0.0,5272.161128,1397.12,5000.00,272.16,0.0,0.000,0.00,NaN


In [1905]:
df1_obj=df.select_dtypes(include=['object']).copy
df1_obj

<bound method NDFrame.copy of Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[24999 rows x 0 columns]>

In [1906]:
df1

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,4200,36,0.116,128.96,6.0,1,69000.0,-1,3,11.08,...,0.0,0.0,4642.374867,4642.37,4200.00,442.37,0.0,0.000,132.82,0.0
1,1325,36,0.119,44.96,0.0,1,24000.0,0,2,3.80,...,0.0,0.0,1476.640000,1476.64,1145.61,287.60,28.5,0.285,44.96,0.0
2,10000,36,0.464,317.72,4.0,-2,75000.0,-1,0,12.64,...,0.0,0.0,10287.347790,10261.63,10000.00,287.35,0.0,0.000,9337.10,0.0
3,6000,60,0.602,147.48,10.0,1,60000.0,0,0,13.06,...,0.0,0.0,8848.343924,8848.34,6000.00,2848.34,0.0,0.000,147.02,0.0
4,25000,36,0.557,869.21,4.0,1,135000.0,0,0,15.49,...,0.0,0.0,31293.587380,31293.59,25000.00,6293.59,0.0,0.000,960.31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,5400,36,0.165,168.60,0.0,1,8000.0,-1,4,3.00,...,0.0,0.0,6069.341452,224.80,5400.00,669.34,0.0,0.000,169.09,NaN
14714,3000,36,0.078,94.98,0.0,-2,25000.0,-1,3,14.54,...,0.0,0.0,3419.144850,1082.73,3000.00,419.14,0.0,0.000,97.45,NaN
14715,5000,36,0.263,156.84,0.0,-2,180000.0,-1,2,5.55,...,0.0,0.0,5645.957239,169.38,5000.00,645.96,0.0,0.000,157.31,NaN
14716,5000,36,0.194,156.84,0.0,-2,100000.0,-1,0,2.30,...,0.0,0.0,5272.161128,1397.12,5000.00,272.16,0.0,0.000,0.00,NaN


In [1907]:
df1.fillna(method ='pad') #Filled all null values with previous one

,loan_amnt,term,int_rate,installment,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,...,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
0,4200,36,0.116,128.96,6.0,1,69000.0,-1,3,11.08,...,0.0,0.0,4642.374867,4642.37,4200.00,442.37,0.0,0.000,132.82,0.0
1,1325,36,0.119,44.96,0.0,1,24000.0,0,2,3.80,...,0.0,0.0,1476.640000,1476.64,1145.61,287.60,28.5,0.285,44.96,0.0
2,10000,36,0.464,317.72,4.0,-2,75000.0,-1,0,12.64,...,0.0,0.0,10287.347790,10261.63,10000.00,287.35,0.0,0.000,9337.10,0.0
3,6000,60,0.602,147.48,10.0,1,60000.0,0,0,13.06,...,0.0,0.0,8848.343924,8848.34,6000.00,2848.34,0.0,0.000,147.02,0.0
4,25000,36,0.557,869.21,4.0,1,135000.0,0,0,15.49,...,0.0,0.0,31293.587380,31293.59,25000.00,6293.59,0.0,0.000,960.31,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14713,5400,36,0.165,168.60,0.0,1,8000.0,-1,4,3.00,...,0.0,0.0,6069.341452,224.80,5400.00,669.34,0.0,0.000,169.09,1.0
14714,3000,36,0.078,94.98,0.0,-2,25000.0,-1,3,14.54,...,0.0,0.0,3419.144850,1082.73,3000.00,419.14,0.0,0.000,97.45,1.0
14715,5000,36,0.263,156.84,0.0,-2,180000.0,-1,2,5.55,...,0.0,0.0,5645.957239,169.38,5000.00,645.96,0.0,0.000,157.31,1.0
14716,5000,36,0.194,156.84,0.0,-2,100000.0,-1,0,2.30,...,0.0,0.0,5272.161128,1397.12,5000.00,272.16,0.0,0.000,0.00,1.0


In [1908]:
df1.isnull().sum() #checking columns which has null 

loan_amnt                      0
term                           0
int_rate                      21
installment                    0
emp_length                  1034
home_ownership                 0
annual_inc                     0
verification_status            0
purpose                        0
dti                            0
delinq_2yrs                    0
inq_last_6mths                 0
mths_since_last_delinq      9442
mths_since_last_record     13611
open_acc                       0
pub_rec                        0
revol_util                    21
total_acc                      0
out_prncp                      0
out_prncp_inv                  0
total_pymnt                    0
total_pymnt_inv                0
total_rec_prncp                0
total_rec_int                  0
recoveries                     0
collection_recovery_fee        0
last_pymnt_amnt                0
pub_rec_bankruptcies         280
dtype: int64

In [1909]:
df1['pub_rec_bankruptcies'].fillna(int(df1['pub_rec_bankruptcies'].mean()), inplace=True)
df1['int_rate'].fillna(int(df1['int_rate'].mean()), inplace=True)
df1['emp_length'].fillna(int(df1['emp_length'].mean()), inplace=True)
df1['mths_since_last_delinq'].fillna(int(df1['mths_since_last_delinq'].mean()), inplace=True)
df1['mths_since_last_record'].fillna(int(df1['mths_since_last_record'].mean()), inplace=True)
df1['revol_util'].fillna(int(df1['revol_util'].mean()), inplace=True)


In [1910]:
df1.isnull().sum() #checking columns which has null 

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length                 0
home_ownership             0
annual_inc                 0
verification_status        0
purpose                    0
dti                        0
delinq_2yrs                0
inq_last_6mths             0
mths_since_last_delinq     0
mths_since_last_record     0
open_acc                   0
pub_rec                    0
revol_util                 0
total_acc                  0
out_prncp                  0
out_prncp_inv              0
total_pymnt                0
total_pymnt_inv            0
total_rec_prncp            0
total_rec_int              0
recoveries                 0
collection_recovery_fee    0
last_pymnt_amnt            0
pub_rec_bankruptcies       0
dtype: int64

In [1911]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
df_train=sc.fit_transform(df)

df_test=sc.fit_transform(df1)
df_train.shape,df_test.shape


((24999, 28), (14718, 28))

In [1912]:
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pylab as ply
model_gb = GradientBoostingClassifier(n_estimators=10,learning_rate=0.5,
                                       max_depth=3,
                                       subsample=0.5,
                                       max_features='log2',
                                       verbose=1)
model_gb.fit(df_train,train_labels)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.2268           0.3020            0.72s
         2           0.1524           0.0715            0.70s
         3           0.1072           0.0445            0.69s
         4           0.0967           0.0169            0.56s
         5           0.0745           0.0167            0.46s
         6           0.0561           0.0181            0.36s
         7           0.0474           0.0067            0.27s
         8           0.0398           0.0045            0.18s
         9           0.0422           0.0024            0.09s
        10           0.0365           0.0019            0.00s


GradientBoostingClassifier(learning_rate=0.5, max_features='log2',
                           n_estimators=10, subsample=0.5, verbose=1)

In [1913]:
len(model_gb.estimators_)

10

In [1914]:
y_pred=model_gb.predict(df_test)

In [1915]:
from sklearn.metrics import accuracy_score
import sklearn.metrics
print(accuracy_score(test_labels,y_pred),sklearn.metrics.precision_score(test_labels, y_pred, average = 'weighted'),sklearn.metrics.recall_score(test_labels, y_pred, average = 'weighted') )

0.9853240929474113 0.9852347970996227 0.9853240929474113


In [1916]:
model_gb1 = GradientBoostingClassifier(n_estimators=400,learning_rate=0.08,
                                       max_depth=3,
                                       subsample=0.5,
                                       max_features='log2',
                                       verbose=1)
model_gb1.fit(df_train,train_labels)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.4047           0.1235           31.75s
         2           0.3580           0.0429           34.50s
         3           0.3391           0.0192           36.36s
         4           0.2978           0.0383           36.49s
         5           0.2784           0.0238           37.36s
         6           0.2557           0.0233           37.08s
         7           0.2353           0.0170           36.96s
         8           0.2205           0.0115           36.09s
         9           0.2130           0.0129           36.52s
        10           0.1985           0.0132           35.83s
        20           0.1277           0.0046           36.09s
        30           0.0920           0.0018           34.60s
        40           0.0726           0.0021           33.41s
        50           0.0568           0.0007           32.29s
        60           0.0495           0.0013           31.32s
       

GradientBoostingClassifier(learning_rate=0.08, max_features='log2',
                           n_estimators=400, subsample=0.5, verbose=1)

In [1917]:
y_pred1=model_gb1.predict(df_test)
print(accuracy_score(test_labels,y_pred1),sklearn.metrics.precision_score(test_labels, y_pred1, average = 'weighted'),sklearn.metrics.recall_score(test_labels, y_pred1, average = 'weighted'))

0.9883815735833673 0.9887756495964876 0.9883815735833673


In [1918]:
model_gb2 = GradientBoostingClassifier(n_estimators=800,learning_rate=0.01,
                                       max_depth=5,
                                       subsample=0.5,
                                       max_features='log2',
                                       verbose=1)
model_gb2.fit(df_train,train_labels)

      Iter       Train Loss      OOB Improve   Remaining Time 
         1           0.5047           0.0167            1.82m
         2           0.4918           0.0156            1.75m
         3           0.4844           0.0116            1.72m
         4           0.4717           0.0100            1.71m
         5           0.4661           0.0115            1.66m
         6           0.4532           0.0077            1.67m
         7           0.4415           0.0108            1.65m
         8           0.4360           0.0084            1.65m
         9           0.4230           0.0094            1.67m
        10           0.4193           0.0072            1.69m
        20           0.3545           0.0045            1.72m
        30           0.3134           0.0044            1.72m
        40           0.2735           0.0024            1.72m
        50           0.2478           0.0028            1.69m
        60           0.2192           0.0019            1.64m
       

GradientBoostingClassifier(learning_rate=0.01, max_depth=5, max_features='log2',
                           n_estimators=800, subsample=0.5, verbose=1)

In [1919]:
y_pred2=model_gb1.predict(df_test)
print(accuracy_score(test_labels,y_pred2),sklearn.metrics.precision_score(test_labels, y_pred2, average = 'weighted'),sklearn.metrics.recall_score(test_labels, y_pred2, average = 'weighted'))
print(model_gb1.feature_importances_)

0.9883815735833673 0.9887756495964876 0.9883815735833673
[4.35455736e-02 1.46336144e-02 6.63590050e-04 1.89707595e-02
 4.10964474e-04 1.66868995e-04 6.76574196e-04 2.00809793e-03
 4.99989612e-04 9.96612877e-04 9.05024315e-05 1.29761416e-03
 4.48466932e-04 4.02467240e-04 4.99212744e-04 2.83082955e-04
 1.89268194e-03 9.83088327e-04 9.12782780e-02 6.39309362e-02
 3.92157388e-02 3.18733736e-02 8.65788149e-02 1.00904566e-02
 2.21504055e-01 3.16257325e-01 5.06410503e-02 1.60208818e-04]


Simple decision tree

In [1921]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

clf = DecisionTreeClassifier(criterion="entropy") # Decision tree built using Info gain
clf = clf.fit(df_train,train_labels)

y_pred3 = clf.predict(df_test)
print(clf.fit(df_train,train_labels).score(df_train,train_labels))
print(accuracy_score(test_labels,y_pred3),sklearn.metrics.precision_score(test_labels, y_pred3, average = 'weighted'),sklearn.metrics.recall_score(test_labels, y_pred3, average = 'weighted'))

1.0
0.9943606468270145 0.9943496942824094 0.9943606468270145
